In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-recommendations-database/rating.csv
/kaggle/input/anime-recommendations-database/anime.csv


In [2]:
ratings = pd.read_csv("/kaggle/input/anime-recommendations-database/rating.csv")
anime = pd.read_csv("/kaggle/input/anime-recommendations-database/anime.csv")
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
ratings.shape

(7813737, 3)

In [5]:
anime.shape

(12294, 7)

In [6]:
ratings.describe()

,user_id,anime_id,rating
count,7.813737e+06,7.813737e+06,7.813737e+06
mean,3.672796e+04,8.909072e+03,6.144030e+00
std,2.099795e+04,8.883950e+03,3.727800e+00
min,1.000000e+00,1.000000e+00,-1.000000e+00
25%,1.897400e+04,1.240000e+03,6.000000e+00
50%,3.679100e+04,6.213000e+03,7.000000e+00
75%,5.475700e+04,1.409300e+04,9.000000e+00
max,7.351600e+04,3.451900e+04,1.000000e+01


In [7]:
anime.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [8]:
ratings.rating.value_counts()

 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: rating, dtype: int64

In [9]:
ratings = ratings[ratings['rating'] != -1]
ratings.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [10]:
df = pd.merge(ratings,anime[["anime_id","name","genre"]], left_on = "anime_id", right_on = "anime_id").drop("anime_id", axis = 1)
df.head()

,user_id,rating,name,genre
0,1,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
1,3,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
2,5,2,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
3,12,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
4,14,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"


In [11]:
discard = ["Hentai"] 
df = df[~df.genre.str.contains('|'.join(discard), na=False)]
df.head()

,user_id,rating,name,genre
0,1,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
1,3,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
2,5,2,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
3,12,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
4,14,6,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"


In [12]:
count_rating = df.groupby("name")["rating"].count().sort_values()
count_rating

name
Ore no Sora Keiji-hen                                1
Orchestra                                            1
Osamishi Tani no Wakare Uta                          1
Oshare Majo Love and Berry: Shiawase no Mahou        1
Dareka no Santa Claus                                1
                                                 ...  
Angel Beats!                                     23565
Code Geass: Hangyaku no Lelouch                  24126
Shingeki no Kyojin                               25290
Sword Art Online                                 26310
Death Note                                       34226
Name: rating, Length: 8846, dtype: int64

In [13]:
y = 100
more_than_y_ratings = count_rating[count_rating.apply(lambda x: x >= y)].index
df_y = df[df['name'].apply(lambda x: x in more_than_y_ratings)]

print('Before:', len(df.name.unique()))
print('After:', len(df_y.name.unique()))

Before: 8846
After: 3985


In [14]:
df_recom = df_y.pivot_table(index='user_id', columns='name', values='rating')
df_recom.iloc[:5,:5]

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy
user_id,,,,,
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN


In [15]:
def find_correlation(df, name):
    similar = df.corrwith(df[name])
    similar = pd.DataFrame(similar, columns = ['Corr'])
    similar = similar.sort_values(by = 'Corr', ascending = False)
    
    return similar

In [16]:
anime ='Yuu☆Yuu☆Hakusho'
find_correlation(df_recom, anime).head(20)

,Corr
name,
Yuu☆Yuu☆Hakusho,1.000000
Soratobu Yuureisen,0.867182
Gekitou! Crush Gear Turbo,0.728637
Mashiro-iro Symphony: Airi ga Anata no Kanojo ni!?,0.726091
Nils no Fushigi na Tabi,0.720015
Pretty Rhythm: Aurora Dream,0.706984
One Off,0.694009
Salaryman Kintarou,0.673323
Strike Witches: Operation Victory Arrow,0.671317
